In [1]:
import pandas as pd
import re
import psycopg2

In [2]:
# load data
filepath = '/home/silas/Documents/DataScience Notebooks/scraping/WikiNews/scrape_wikinews.csv'
df = pd.read_csv(filepath,
                names = ['id','title','url','date_written','keywords','content'],
                dtype={'id':'str'})

In [3]:
#str -> list
df['keywords'] =df['keywords'].apply(lambda x: re.findall(r"'([^']+)'",x))

## Create CSV-files

In [4]:
def make_dicts():
    
    def makeDateDict():
        date_lst = list(df['date_written'].unique())
        date_lst.remove('None')
        return {elm:idx for idx,elm in enumerate(date_lst)} 
    
    def makeKeywordDict():
        st = set()
        for elm in df['keywords']:
            if elm != None: st.update(elm)
        return {elm:idx for idx,elm in enumerate(st)}

    return makeDateDict(), makeKeywordDict(),

#get dicts
date_dict, keyword_dict = make_dicts()

In [5]:
# article csv
def make_article_csv():
    article_df = pd.DataFrame(
        data={'id':list(df.id),
              'time_id':list(map(lambda x: str(date_dict[x]) if x != "None" else None,list(df.date_written)))
             }
                            )
    article_df = article_df.join(df[['id','url','content','title']].set_index('id'),on='id')
    article_df.to_csv('wiki_article.csv',index=False)

In [6]:
def make_simple_csv(dictx,filename):
    frame = pd.DataFrame({'id':list(dictx.values()), 'value':list(dictx)})
    frame.drop_duplicates().to_csv(filename,index=False)

In [7]:
def make_multi_csv(dictx,filename,column_name,id_name):
    frame = pd.DataFrame(df[['id',column_name]])
    frame = frame.dropna()
    frame = frame.explode(column_name)
    frame[column_name] = frame[column_name].apply(lambda x: dictx[x])
    frame.drop_duplicates().to_csv(filename, index=False, header=['id',id_name])

In [8]:
make_article_csv()

make_simple_csv(date_dict,'wiki_time.csv')
make_simple_csv(keyword_dict,'wiki_keyword.csv')

make_multi_csv(keyword_dict,'wiki_keywords.csv','keywords','keyword_id')

## Load data into database

In [9]:
def execQuery(query,*printOption):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "1234",
                                      host = "localhost",
                                      port = "5432",
                                      database = "FakeNews")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL:", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            if not printOption:
                print("Executed query and closed connection.")

In [10]:
queryCreate = """
DROP TABLE IF EXISTS wiki_time CASCADE;
DROP TABLE IF EXISTS wiki_article CASCADE;
DROP TABLE IF EXISTS wiki_keywords CASCADE;
DROP TABLE IF EXISTS wiki_keyword;

CREATE TABLE wiki_time(
    time_id int PRIMARY KEY,
    time_stamp timestamp);

CREATE TABLE wiki_article(
    article_id int PRIMARY KEY,
    time_id int REFERENCES wiki_time,
    url varchar,
    content varchar,
    title varchar);
    
CREATE TABLE wiki_keyword(
    keyword_id int PRIMARY KEY,
    value varchar
    );

CREATE TABLE wiki_keywords(
    article_id int REFERENCES wiki_article,
    keyword_id int REFERENCES wiki_keyword,
    PRIMARY KEY(article_id,keyword_id)
    );
    
COPY wiki_time FROM '/home/silas/Documents/DataScience Notebooks/csv/wikiNews/wiki_time.csv' WITH (FORMAT csv, HEADER true);
COPY wiki_article FROM '/home/silas/Documents/DataScience Notebooks/csv/wikiNews/wiki_article.csv' WITH (FORMAT csv, HEADER true);
COPY wiki_keyword FROM '/home/silas/Documents/DataScience Notebooks/csv/wikiNews/wiki_keyword.csv' WITH (FORMAT csv, HEADER true);
COPY wiki_keywords FROM '/home/silas/Documents/DataScience Notebooks/csv/wikiNews/wiki_keywords.csv' WITH (FORMAT csv, HEADER true);
"""

execQuery(queryCreate)

Error while connecting to PostgreSQL: no results to fetch


This result is expected since the query creates and inserts, but does not return anything.

In [11]:
#demonstrate database

queryDemonstrate = """
SELECT count(distinct article_id) 
FROM wiki_time RIGHT JOIN
    wiki_article using (time_id) INNER JOIN
    wiki_keywords using (article_id) INNER JOIN
    wiki_keyword using (keyword_id)
LIMIT 5"""

execQuery(queryDemonstrate)

Executed query and closed connection.


[(7474,)]